In [1]:
import librosa
import numpy as np
import pandas as pd
import json
import requests
import os

In [2]:
json_file_path = "./speechcrawling/records.json"
with open(json_file_path, 'r') as file:
    data = json.load(file)
df = pd.DataFrame(data)
df.head()

,original_url,record
0,https://freesound.org/people/kevp888/sounds/71...,https://cdn.freesound.org/previews/712/712632_...
1,https://freesound.org/people/Robinhood76/sound...,https://cdn.freesound.org/previews/712/712628_...
2,https://freesound.org/people/josefpres/sounds/...,https://cdn.freesound.org/previews/712/712630_...
3,https://freesound.org/people/newlocknew/sounds...,https://cdn.freesound.org/previews/712/712629_...
4,https://freesound.org/people/josefpres/packs/3...,https://cdn.freesound.org/previews/712/712630_...


In [3]:
def extract_record_name(url):
    url_sections= url.split("/")
    record = url_sections[-1]
    return record

In [4]:
def download_and_save(url):
    mp3_record = extract_record_name(url)
    file_path = "./mp3_records/" + mp3_record

    if os.path.exists(file_path):
        print(f"The file {mp3_record} already exists. Skipping download!")
    else:
        response = requests.get(url)
        if response.status_code == 200:
            with open(file_path, "wb") as file:
                file.write(response.content)
            print(f"File {mp3_record} downloaded successfully.")
        else:
            print(f"Failed to download  {mp3_record}. Status code {response.status_code}")

In [5]:
for url in df['record']:
    download_and_save(url)


The file 712632_9034501-lq.mp3 already exists. Skipping download!
The file 712628_321967-lq.mp3 already exists. Skipping download!
The file 712630_5674468-lq.mp3 already exists. Skipping download!
The file 712629_5828667-lq.mp3 already exists. Skipping download!
The file 712630_5674468-lq.mp3 already exists. Skipping download!
The file 676694_5674468-lq.mp3 already exists. Skipping download!
The file 660950_5674468-lq.mp3 already exists. Skipping download!
The file 656816_5674468-lq.mp3 already exists. Skipping download!
The file 654671_5674468-lq.mp3 already exists. Skipping download!
The file 652687_5674468-lq.mp3 already exists. Skipping download!
The file 646029_5674468-lq.mp3 already exists. Skipping download!
The file 644491_5674468-lq.mp3 already exists. Skipping download!
The file 712627_1089955-lq.mp3 already exists. Skipping download!
The file 712628_321967-lq.mp3 already exists. Skipping download!
The file 702515_321967-lq.mp3 already exists. Skipping download!
The file 6338

In [20]:
directory_path = './mp3_records'

files_and_directories = os.listdir(directory_path)
records_list=[]
# Print only the files
for item in files_and_directories:
    full_path = os.path.join(directory_path, item)
    if os.path.isfile(full_path):
        records_list.append(item)

In [7]:
def estimate_noise(signal, speech_estimate):
    # Estimate noise by subtracting the estimated speech from the original signal
    noise_estimate = signal - speech_estimate
    return noise_estimate

In [8]:
def calculate_snr(rms_audio, rms_noise):
    snr = 20 * np.log10( rms_audio / rms_noise )
    return snr

In [9]:
def calculate_rms(signal):
    return np.sqrt(np.mean(signal**2))

In [10]:
def estimate_audio_quality(file_path):
    # Load audio file
    audio, sample_rate = librosa.load(file_path, sr=None)

    # Calculate audio RMS
    rms_audio = calculate_rms(audio)

    # Calculate Speech Level
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sample_rate)[0]
    speech_level = np.mean(spectral_centroid)

    # Assuming a simple speech estimate
    speech_estimate = librosa.effects.preemphasis(audio)

    # Estimate noise
    noise_estimate = estimate_noise(audio, speech_estimate)

    # Calculate noise RMS
    rms_noise = calculate_rms(noise_estimate)

    # Calculate SNR ratio
    snr=calculate_snr(rms_audio,rms_noise)

    return speech_level, snr

In [14]:
df_dic={}

In [19]:
df['record'][0]

'https://cdn.freesound.org/previews/712/712632_9034501-lq.mp3'

In [18]:
a,v=estimate_audio_quality('./mp3_records/157321_2305278-lq.mp3')

In [21]:
data_list=[]
for record in records_list:
    quality_info = estimate_audio_quality('./mp3_records/' + record)
    data_list.append({'record': record, 'speech_level': quality_info[0], 'snr': quality_info[1]})

c:\Users\rimai\Desktop\Data-Engineering-Working-Student-Position\venv\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=557
  warnings.warn(


In [22]:
data=pd.DataFrame(df_dic) 

In [25]:
data.head(3)

,157321_2305278-lq.mp3,157326_2305278-lq.mp3,160229_288-lq.mp3,160230_288-lq.mp3,160231_288-lq.mp3,160232_288-lq.mp3,160233_288-lq.mp3,160234_288-lq.mp3,160235_288-lq.mp3,160236_288-lq.mp3,...,712628_321967-lq.mp3,712629_5828667-lq.mp3,712630_5674468-lq.mp3,712632_9034501-lq.mp3,9154_288-lq.mp3,9155_288-lq.mp3,9156_288-lq.mp3,9157_288-lq.mp3,9158_288-lq.mp3,9159_288-lq.mp3
speech_level,4783.927389,4195.299024,2693.866885,6282.672665,6608.544751,3164.411303,1918.361696,2028.203156,2033.382331,6539.605251,...,1741.597092,3972.296441,682.239140,1606.171186,3148.797365,3057.816615,2981.780100,2927.007814,2636.605678,3325.751229
snr,0.264565,0.264564,0.264564,0.264563,0.264562,0.264563,0.264565,0.264564,0.264570,0.264537,...,0.264565,0.264565,0.264567,0.264566,0.264567,0.264564,0.264566,0.264567,0.264566,0.264565


In [23]:
data.to_csv('records.csv',index=False)